In [4]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None

In [5]:
import csv
with open('drugs.csv', 'r') as g:
    reader = csv.reader(g)
    drugs = list(reader)[0]

In [6]:
f = 'acutis_current.csv'
df=pd.read_csv(f,low_memory=False)
df['COLLECT_DATE']=pd.to_datetime(df['COLLECT_DATE'],infer_datetime_format=True,errors='coerce')

done reading
done fixing dates


In [13]:
df=df[df['COLLECT_DATE'] >= pd.to_datetime('04/01/2019',infer_datetime_format=True)].copy()

In [14]:
def createname(row):
    fname = row['PATIENT_F_NAME']
    lname = row['PATIENT_L_NAME']
    name = str(lname)+', '+fname
    return name

df['PATIENT_NAME']=df.apply(createname,axis=1)

In [15]:
df1=df.copy()
df2=df.copy()
df2.set_index(['PATIENT_NAME'],inplace=True)
df2.sort_index(inplace=True)

In [16]:
df1=df1[['PATIENT_NAME','COLLECT_DATE','TEST_NAME','RESULT_FLAG','RESULT_NUMERIC']]
df2=df2[['COLLECT_DATE']]

In [17]:
#We want to get a flag if the sample is the first one on record
lff=[]
for it in df1.itertuples():
    name=it.PATIENT_NAME
#     print(name)
    try:
        earl=np.min(df2.loc[name,'COLLECT_DATE'])
    except TypeError:
        earl=np.min(np.array(df2.loc[name,'COLLECT_DATE']))
#     letd.append(earl)
    if earl==it.COLLECT_DATE:
        lff.append(1)
    else:
        lff.append(0)
arrff=np.array(lff)
df1['FIRST_FLAG']=arrff
del(earl,lff,arrff)

In [28]:
df1[df1['PATIENT_NAME']=='Mayhue, Anthony']

,PATIENT_NAME,COLLECT_DATE,TEST_NAME,RESULT_FLAG,RESULT_NUMERIC,FIRST_FLAG
157768,"Mayhue, Anthony",2019-05-13 06:20:00,Gabapentin,Negative,NaN,1
157769,"Mayhue, Anthony",2019-05-13 06:20:00,Pregabalin,Negative,NaN,1
157770,"Mayhue, Anthony",2019-05-13 06:20:00,Buprenorphine,Positive,337.000,1
157771,"Mayhue, Anthony",2019-05-13 06:20:00,Naloxone,Positive,2574.000,1
157772,"Mayhue, Anthony",2019-05-13 06:20:00,Norbuprenorphine,Positive,1236.000,1
157773,"Mayhue, Anthony",2019-05-13 06:20:00,Creatinine,NaN,148.000,1
157774,"Mayhue, Anthony",2019-05-13 06:20:00,Specific Gravity,NaN,1.019,1
157775,"Mayhue, Anthony",2019-05-13 06:20:00,pH,NaN,7.000,1


In [19]:
#We want rows who are on the date of the first urine sample for the patient, are testing for substances on the list of drugs, 
# and are on or after the first of June (so that way we know its actually the first sample and there likely wasnt one before May 1)
dffirst=df1[(df1['FIRST_FLAG']==1) & (df1['TEST_NAME'].isin(drugs)) & (df1['COLLECT_DATE'] >= pd.to_datetime('June 1, 2018',infer_datetime_format=True))].copy()
dffirst.set_index(['PATIENT_NAME','RESULT_FLAG'],inplace=True)
dffirst.sort_index(inplace=True)

In [20]:
#sort names into lists, save these lists to csvs
d=[]
nd=[]
for i in range(dffirst.shape[0]):
    name=dffirst.index[i][0]
    try:
        dffirst.loc[name,'Positive']
        d.append(name)
    except KeyError:
        nd.append(name)
da=np.array(d)
nda=np.array(nd)
da=np.unique(da)
nda=np.unique(nda)
allnames=np.unique(df['PATIENT_NAME'])
daandnda=np.concatenate((nda,da))
others=np.setdiff1d(allnames,daandnda)

In [21]:
allnames.shape[0]

973

In [22]:
posdf=pd.DataFrame({'Patient':da,'Status':np.array(['Positive']*da.shape[0])})
negdf=pd.DataFrame({'Patient':nda,'Status':np.array(['Negative']*nda.shape[0])})
result=pd.concat([posdf,negdf])
result.sort_values('Patient',inplace=True)
result.reset_index(drop=True,inplace=True)

In [23]:
df1['Patient'
]

,Patient,Status
0,"Adams, Joanne",Positive
1,"Adams, Stephanie",Positive
2,"Adkins, Heath",Positive
3,"Ajello, Dominic",Positive
4,"Alderson, Dennis",Negative
5,"Allen, Patrick",Positive
6,"Altman, Lea",Negative
7,"Anders, Katie",Positive
8,"Andrews, Steven",Positive
9,"Ash, Elisabeth",Positive


In [48]:
fo='PosAndNegUrine.csv'
path='C:\\Users\\Staff\\Google Drive\\LP\\NEW Intake Form Cleaning\\merging\\'
result.to_csv(fo,index=False)
result.to_csv(path+fo,index=False)